Created: 7/1/2025

Purpose: Exploratory Data analysis on the generated model dataset

In [1]:
# import libraries and functions

from heatmap_analysis import Report, gather_model_paths, load_references, open_log, step_file_system
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

This is the setup to load dataset into memory for further analysis

In [2]:
class_path = "D:\\model_dataset\\classification"
regress_path = "D:\\model_dataset\\regression"
model_paths = gather_model_paths(class_path, regress_path)
class_models = load_references(model_paths[1])
regress_models = load_references(model_paths[0])
class_logs_agg = []
regress_logs_agg = []
class_by_dataset = {}
regress_by_dataset = {}

# Gather the data for the classification models
for model in class_models:
    log = open_log(model.model_path)
    class_logs_agg.append(log)
    rows = model.model_path.split('\\')[-2]
    if rows in class_by_dataset:
        class_by_dataset[rows].append(log)
    else:
        class_by_dataset.update({rows: [log]})

#Gather the data fro the regression models
for model in regress_models:
    log = open_log(model.model_path)
    regress_logs_agg.append(log)
    rows = model.model_path.split('\\')[-2]
    if rows in regress_by_dataset:
        regress_by_dataset[rows].append(log)
    else:
        regress_by_dataset.update({rows: [log]})

In [4]:
class_data = pd.DataFrame(class_by_dataset)
class_agg = pd.Series(class_logs_agg)
regress_agg = pd.Series(regress_logs_agg)
regress_data = pd.DataFrame(regress_by_dataset)
class_acc = class_data.copy().map(lambda x: (x.accuracy, x.optimizer))
regress_acc = regress_data.copy().map(lambda x: (x.accuracy, x.optimizer))

In [8]:
temp_agg = class_agg.copy().to_numpy()
temp_r_agg = regress_agg.copy().to_numpy()
optimizers_dict = {}
for index, item in enumerate(temp_r_agg):
    class_opt = temp_agg[index].optimizer
    if item.optimizer in list(optimizers_dict.keys()):
        optimizers_dict[item.optimizer] += 1
    else:
        optimizers_dict.update({item.optimizer: 1})
    if class_opt in list(optimizers_dict.keys()):
        optimizers_dict[class_opt] += 1
    else:
        optimizers_dict.update({class_opt: 1})
print(optimizers_dict)

{"<class 'torch.optim.sgd.SGD'>": 404, "<class 'torch.optim.adam.Adam'>": 390, "<class 'torch.optim.asgd.ASGD'>": 386, "<class 'torch.optim.rmsprop.RMSprop'>": 420}


Note: Seems the dataset is biased towards the RMSProp optimizer, but not by a huge margin.